## Conversation AI Chabot using Gradio

Gradio is an open-source Python package that allows you to quickly build a demo or web application for your machine learning model, API, or any arbitrary Python function. You can then share a link to your demo or web application in just a few seconds using Gradio's built-in sharing features.

In [23]:
import os
from dotenv import load_dotenv
import openai
import gradio as gr

In [24]:
load_dotenv()
openai_api = os.getenv('OPENAI_API_KEY')

if openai_api:
    print(f"OpenAI API key exists and begins {openai_api[:8]}")
else: 
    print("OpenAI API key is not set. Fix it and try again.")

OpenAI API key exists and begins sk-proj-


In [25]:
openAI = OpenAI()
MODEL = 'gpt-4o-mini'

We will write a function chat(message, history) where:
message is the prompt to use
history is the past conversation, in OpenAI format

We will combine the system message, history and latest message, then call OpenAI.

## DeutschMeister: Your AI German Language Coach"

In [26]:
system_prompt = """
You are a professional German language educator. Your role is to help users learn, practice, and master the German language. 
You provide lessons, explanations, practice exercises, and exam preparation strategies — especially for standardized German language tests 
like the Goethe-Zertifikat, TestDaF, or TELC. 

Your response format must be clear and organized using markdown. Include:
- Headings for grammar topics
- Bullet points or numbered steps for explanations
- Tables for grammar rules (e.g., article declensions, verb conjugations)
- Code blocks for example sentences (wrapped in triple backticks)
- Bold for key concepts
- Italics for translations
- Quizzes with blanks or multiple choice
- Encourage the user to respond or ask questions

Tailor your response to the user's selected level (A1–C2) and chosen mode:
- "Lesson Mode": Provide structured lessons with examples
- "Practice Mode": Give interactive exercises and corrections
- "Exam Prep Mode": Simulate parts of German proficiency exams

If the user asks something unrelated to the German language, politely refuse and guide them back.

Be encouraging and structured like a real tutor. Offer learning plans if asked, and tailor content to the user’s level (A1–C2).
"""

In [31]:
def chat_with_ai(message, history, level, mode):
    """
    Handles a user message by sending it to the OpenAI Chat API along with chat history and system instructions.
    """
    
    custom_prompt = system_prompt +  f"\n\nThe user is at level {level} and selected {mode} mode.\n"
    messages = [{"role": "system", "content": custom_prompt}]

    for item in history:
        if isinstance(item, dict):
            messages.append(item)
        elif isinstance(item, (list, tuple)) and len(item) == 2:
            user_msg, bot_reply = item
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": bot_reply})
            
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

    reply = response.choices[0].message.content
    history.append((message, reply))
    
    chatbot_messages = []
    for user_msg, bot_msg in history:
        chatbot_messages.append({"role": "user", "content": user_msg})
        chatbot_messages.append({"role": "assistant", "content": bot_msg})

    return chatbot_messages, history

In [32]:
with gr.Blocks(title = "DeutschMeister: Your AI German Language Coach") as demo:
    gr.Markdown("## 🇩🇪 **DeutschMeister**\n*Your AI German Language Coach for Exams, Lessons & Practice*")

    with gr.Row():
        level = gr.Dropdown(["A1", "A2", "B1", "B2", "C1", "C2"], label = "Your German Level", value = "A1")
        mode = gr.Radio(["Lesson Mode", "Practice Mode", "Exam Prep Mode"], label = "Select Mode", value = "Lesson Mode")

    chatbot = gr.Chatbot(label = "DeutschMeister", render_markdown = True, type = "messages")
    msg = gr.Textbox(label = "Ask a question, request a lesson, or type an exercise...")
    clear = gr.Button("🧹 Clear Chat")

    history = gr.State([])

    msg.submit(chat_with_ai, [msg, history, level, mode], [chatbot, history])
    clear.click(lambda: ([], []), outputs = [chatbot, history])

demo.launch(inline = True)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.
